## This notebook compare the differents peaks search

In [ ]:
%matplotlib inline

import multiprocessing.pool
import timeit

import torch

from laueimproc import Diagram
from laueimproc.io import get_samples

### Firstly, we have to create the diagrams from images
* To see all the ways to create a diagram, please have a look to the `init_diagram` notebook.
* Take care having enough RAM and swappiness to load all the images, otherwise it would bias the peak search time.

In [ ]:
all_files = sorted(get_samples().iterdir())  # the list of all images path
diagrams = [Diagram(f) for f in all_files]

In [ ]:
def load_images():
    with multiprocessing.pool.ThreadPool() as pool:
        pool.map(lambda d: (d.compress(), d.image), diagrams)
times = timeit.repeat(load_images, repeat=3, number=1)
print(f"on average, an image is read in {1000*min(times)/len(diagrams):.2f}ms")

### Secondly, prepare tools for spots evaluation

In [ ]:
def stats_peaks_search(**kwargs):
    def run():
        with multiprocessing.pool.ThreadPool() as pool:
            pool.map(lambda d: d.find_spots(**kwargs), diagrams)
    times = timeit.repeat(run, repeat=3, number=1)
    print(f"average stats for find_spots({', '.join(f'{k}={v}' for k, v in kwargs.items())}):")
    print(f"    time: {1000*min(times)/len(diagrams):.2f}ms/diag")
    nb_spots = torch.tensor([len(d) for d in diagrams], dtype=torch.float32)
    # print(f"    nbr spots mean: {float(torch.mean(nb_spots)):.1f}")
    print(f"    nbr spots median: {float(torch.median(nb_spots)):.1f}")
    intensities = torch.cat([d.compute_rois_sum(cache=False) for d in diagrams])
    # print(f"    intensity mean: {float(torch.mean(intensities)):.3f}")
    print(f"    intensity median: {float(torch.median(intensities)):.3f}")
    # print(f"    intensity std: {float(torch.std(intensities)):.3f}")

### Thirdly, run the peaks search

In [ ]:
for density in (0.9, 0.8, 0.7, 0.6, 0.55, 0.5, 0.45, 0.4, 0.3, 0.2, 0.1):
    stats_peaks_search(density=density)
    print()
for threshold in (1e-5, 3.2e-5, 1e-4, 3.2e-4, 1e-3, 3.2e-3, 1e-2):
    stats_peaks_search(threshold=threshold)
    print()
for radius_font in (4, 5, 6, 7, 8, 9, 10, 11, 12, 13):
    stats_peaks_search(radius_font=radius_font)
    print()
for radius_aglo in (1, 2, 3, 4, 5):
    stats_peaks_search(radius_aglo=radius_aglo)
    print()